In [1]:
import pandas as pd
import os
from implicit.als import AlternatingLeastSquares
import numpy as np

# 1) 데이터 준비와 전처리

Movielens 데이터를 불러온다.

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


Movielens 데이터에는 user_id, movie_id, 평점, 타임스탬프가 기록 되어있는 것을 확인할 수 있다.

In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외한다.

In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
ratings

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


ratings 열이 직관적으로 count 열로 바뀌었으며, 각 유저가 어떤 영화에 몇 점을 주었는지 확인할 수 있다.

In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


위 movie 데이터 프레임은 각 movie_id에 대응되는 영화가 어떤 제목이고 어떤 장르인지 확인할 수 있게 되었다.

여기까지가 전처리이고, 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있다.

ratings 데이터프레임에 movies 데이터 프레임을 movie_id 기준으로 join해야한다.

In [8]:
ratings = pd.merge(ratings, movies, on='movie_id', how='left')
ratings

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...
836473,6040,1090,3,956715518,Platoon (1986),Drama|War
836474,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama
836476,6040,1096,4,956715648,Sophie's Choice (1982),Drama


In [9]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'title', 'counts']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,title,counts
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,1,James and the Giant Peach (1996),3
2,1,My Fair Lady (1964),3
3,1,Erin Brockovich (2000),4
4,1,"Bug's Life, A (1998)",5
5,1,"Princess Bride, The (1987)",3
6,1,Ben-Hur (1959),5
7,1,"Christmas Story, A (1983)",5
8,1,Snow White and the Seven Dwarfs (1937),4
9,1,"Wizard of Oz, The (1939)",4


정해진 컬럼만 남은 것이 보인다.

# 2) 분석해 봅시다.

- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

pandas.DataFrame.nunique()은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용하다.

In [10]:
# ratings에 있는 유니크한 영화 개수
ratings['title'].nunique()

3628

In [11]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [12]:
# 가장 인기있는 영화 30개 (인기순)
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

가장 인기있는 영화는 American Beauty로 3211표인 것을 확인할 수 있었다.

# 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.


In [13]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['Toy Story (1995)' , 'Jumanji (1995)' ,'Grumpier Old Men (1995)' ,'Waiting to Exhale (1995)' ,'Father of the Bride Part II (1995)']

# '6041'이라는 user_id가 위 영화들을 5점을 주었다고 보았다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': [6041]*5, 'title': my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():  # user_id에 'kang'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,title,counts
836473,6040,Platoon (1986),3
836474,6040,"Crying Game, The (1992)",5
836475,6040,Welcome to the Dollhouse (1995),5
836476,6040,Sophie's Choice (1982),4
836477,6040,E.T. the Extra-Terrestrial (1982),4
0,6041,Toy Story (1995),5
1,6041,Jumanji (1995),5
2,6041,Grumpier Old Men (1995),5
3,6041,Waiting to Exhale (1995),5
4,6041,Father of the Bride Part II (1995),5


이제 모델에 활용하기 이전 indexing을 진행한다.

In [14]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

print(user_unique)

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

[   1    2    3 ... 6039 6040 6041]


In [15]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx[6041])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(movie_to_idx['Toy Story (1995)'])

6039
40


In [17]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # ratings['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,counts
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,40,5
1,6039,513,5
2,6039,1862,5
3,6039,397,5


# 4) CSR matrix를 직접 만들어 봅시다.


In [18]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.


In [19]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [28]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

여기서 gpu를 True로 하면 이후 벡터를 내적하는 코드에서 동작하지 않는다.

In [29]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [30]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

나의 아이디는 6041, 내가 선호하는 영화중 중 Toy Story를 선택해 그 벡터곱을 확인한다.

In [31]:
my_id, toy_story = user_to_idx[6041], movie_to_idx['Toy Story (1995)']
my_id_vector, toy_story_vector = als_model.user_factors[my_id], als_model.item_factors[toy_story]

In [32]:
my_id_vector

array([-0.4412221 , -0.3609956 , -0.30594483, -0.10939153, -0.19388402,
        0.32674792,  0.41967714,  0.38358366, -0.142182  , -0.6887313 ,
        0.06773965,  0.219439  ,  0.38595772, -0.23745938, -0.517639  ,
        0.3969938 , -0.08777852,  0.90860873, -0.40708596,  0.3236135 ,
        0.87075496, -0.41871226,  0.3161251 , -0.7624456 , -0.6908729 ,
       -0.09870125, -0.32357514,  0.29226896,  0.13027763,  0.36445636,
       -0.02343026,  0.41535035,  0.16655567, -0.1035177 ,  0.2005209 ,
        0.23094545,  0.22147338,  0.15995307, -0.29258963,  0.46466026,
       -0.5962454 , -0.28575355,  0.35371748,  0.07529598, -0.05410279,
        0.34789997, -0.55053353,  0.08261206, -0.06061924,  0.22658822,
       -0.52370244, -0.4780919 , -0.35986507,  0.37782076, -0.41206217,
       -0.6391383 ,  0.7074855 , -0.30382404,  1.0395547 ,  0.48839796,
        0.28800613,  0.44196084, -0.18883532, -0.02605919, -0.06741276,
        0.2233813 ,  0.31237608,  0.6820395 , -0.59479994, -0.56

In [33]:
toy_story_vector

array([-3.37346061e-03, -7.69280875e-03, -2.95162220e-02, -2.16067769e-03,
        1.98903270e-02,  2.93388721e-02,  1.16056325e-02,  2.29434557e-02,
       -1.05669042e-02,  6.50810252e-05,  3.94543484e-02, -1.08293351e-02,
       -1.53062108e-03,  2.00624987e-02,  1.28358053e-02,  1.48527352e-02,
        8.29063449e-03,  3.38303894e-02, -9.63192992e-03, -1.44631006e-02,
        4.02154103e-02,  9.39194765e-03,  1.79576036e-02, -2.57005841e-02,
        1.12420497e-02,  9.79293138e-04, -2.54703462e-02,  3.39399208e-03,
       -6.57180883e-03,  1.13324123e-02,  9.18210391e-03,  1.64984427e-02,
        5.34325233e-03, -1.07324813e-02,  2.34104190e-02, -2.68774596e-03,
        3.66797671e-03, -2.16227467e-03, -2.69641690e-02,  5.86048188e-03,
       -1.88202914e-02,  2.19700783e-02,  3.07935718e-02, -2.47476902e-03,
        2.50462834e-02, -7.12312106e-03,  7.25955982e-03,  3.24653722e-02,
        4.93239099e-03, -3.19076446e-03, -3.81502062e-02, -8.47724325e-04,
        1.51641562e-03,  

In [34]:
# my_id와 toy_story를 내적하는 코드
np.dot(my_id_vector, toy_story_vector)

0.4377068

1이 나와야 될 것 같은데 한참 낮은 수치인 0.437 정도가 나왔다. factors를 늘리거나 iterations를 늘려야 할 것 같다.

iterations를 2배 늘려본다.

In [35]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [36]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

다시 내적을 확인해본다.

In [37]:
my_id, toy_story = user_to_idx[6041], movie_to_idx['Toy Story (1995)']
my_id_vector, toy_story_vector = als_model.user_factors[my_id], als_model.item_factors[toy_story]
# my_id와 toy_story를 내적하는 코드
np.dot(my_id_vector, toy_story_vector)

0.46952906

아까보단 개선되었다. iterations를 또 2배로 늘려본다.

In [38]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=60, dtype=np.float32)
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/60 [00:00<?, ?it/s]

In [39]:
my_id_vector, toy_story_vector = als_model.user_factors[my_id], als_model.item_factors[toy_story]
# my_id와 toy_story를 내적하는 코드
np.dot(my_id_vector, toy_story_vector)

0.49106604

이번엔 factors를 늘려본다.

In [40]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=60, dtype=np.float32)
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/60 [00:00<?, ?it/s]

In [41]:
my_id_vector, toy_story_vector = als_model.user_factors[my_id], als_model.item_factors[toy_story]
# my_id와 toy_story를 내적하는 코드
np.dot(my_id_vector, toy_story_vector)

0.7049937

iterations를 두 배 늘릴 때마다 이전보다 약 8% 개선되는 것으로 보이고 <br>
factors를 두 배 늘리니 이전보다 약 40% 개선되었다. 

이번엔 factors와 iterations 모두 늘려본다.

In [42]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=300, regularization=0.01, use_gpu=False, iterations=120, dtype=np.float32)
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/120 [00:00<?, ?it/s]

In [43]:
my_id_vector, toy_story_vector = als_model.user_factors[my_id], als_model.item_factors[toy_story]
# my_id와 toy_story를 내적하는 코드
np.dot(my_id_vector, toy_story_vector)

0.835091

이전 모델보다 약 20% 개선되었다.  

In [44]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=400, regularization=0.01, use_gpu=False, iterations=200, dtype=np.float32)
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/200 [00:00<?, ?it/s]

In [45]:
my_id_vector, toy_story_vector = als_model.user_factors[my_id], als_model.item_factors[toy_story]
# my_id와 toy_story를 내적하는 코드
np.dot(my_id_vector, toy_story_vector)

0.8917621

In [46]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=500, regularization=0.01, use_gpu=False, iterations=300, dtype=np.float32)
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/300 [00:00<?, ?it/s]

In [47]:
my_id_vector, toy_story_vector = als_model.user_factors[my_id], als_model.item_factors[toy_story]
# my_id와 toy_story를 내적하는 코드
np.dot(my_id_vector, toy_story_vector)

0.93020463

이정도면 학습 데이터에 대해서 fitting은 되었다고 볼 수 있을 것이다. 

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.


내가 선호하는 5가지 영화중 Toy Story에 대해서는 선호도를 0.93으로 예측하였다. <br>
이번에는 그 외의 영화를 하나 골라 나의 선호도를 예측해본다. 

In [49]:
my_id, matrix_the = user_to_idx[6041], movie_to_idx['Matrix, The (1999)']
my_id_vector, matrix_the_vector = als_model.user_factors[my_id], als_model.item_factors[matrix_the]
np.dot(my_id_vector, matrix_the_vector)

0.00087570923

훈련된 모델은 내가 매트릭스를 좋아하지 않을 것이라고 판단하였다.

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.


In [51]:
favorite_movie = 'Jumanji (1995)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(513, 0.9999999),
 (1130, 0.35185117),
 (828, 0.3157874),
 (596, 0.3061146),
 (173, 0.29087797),
 (935, 0.24187967),
 (1518, 0.22494495),
 (576, 0.22430882),
 (2017, 0.22329381),
 (1733, 0.21824244),
 (1985, 0.2160244),
 (545, 0.20954624),
 (1982, 0.20205516),
 (1238, 0.2017305),
 (1736, 0.19860598)]

In [52]:
#movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Jumanji (1995)',
 'Indian in the Cupboard, The (1995)',
 'Flubber (1997)',
 'Hook (1991)',
 'Dragonheart (1996)',
 'Casper (1995)',
 'Free Willy (1993)',
 'Escape to Witch Mountain (1975)',
 'Pagemaster, The (1994)',
 'Borrowers, The (1997)',
 'NeverEnding Story II: The Next Chapter, The (1990)',
 'Santa Clause, The (1994)',
 'Space Jam (1996)',
 'Mighty Joe Young (1998)',
 'Small Soldiers (1998)']

쥬만지와 비슷한 영화는 1130번 , 'Indian in the Cupboard, The (1995)'라고 판단하였다.

In [56]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

이번엔 'Grumpier Old Men (1995)'과 유사한 작품을 찾아본다.

In [57]:
get_similar_movie('Grumpier Old Men (1995)')

['Grumpier Old Men (1995)',
 'Grumpy Old Men (1993)',
 'Odd Couple II, The (1998)',
 'Milk Money (1994)',
 'Out to Sea (1997)',
 'Two if by Sea (1996)',
 'Naked Gun 33 1/3: The Final Insult (1994)',
 'Rudy (1993)',
 'Sister Act 2: Back in the Habit (1993)',
 'Son of Flubber (1963)']

그건 'Odd Couple II, The (1998)' 라고 한다.

# 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.


In [58]:
user = user_to_idx[6041]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(1045, 0.3847711),
 (596, 0.21209331),
 (560, 0.1744903),
 (1769, 0.16855085),
 (1165, 0.16776867),
 (828, 0.16635679),
 (567, 0.15535942),
 (341, 0.15494527),
 (1586, 0.14793481),
 (545, 0.14210375),
 (1358, 0.13994206),
 (421, 0.13404885),
 (892, 0.13097876),
 (1516, 0.12896284),
 (499, 0.12809552),
 (639, 0.12765229),
 (1207, 0.12758577),
 (1172, 0.12463925),
 (1982, 0.12428503),
 (173, 0.1231359)]

In [60]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Grumpy Old Men (1993)',
 'Hook (1991)',
 'Nine Months (1995)',
 "City Slickers II: The Legend of Curly's Gold (1994)",
 'Toys (1992)',
 'Flubber (1997)',
 'First Wives Club, The (1996)',
 'Mighty Ducks, The (1992)',
 'Sister Act 2: Back in the Habit (1993)',
 'Santa Clause, The (1994)',
 'Birds, The (1963)',
 'Midnight in the Garden of Good and Evil (1997)',
 'In & Out (1997)',
 'How to Make an American Quilt (1995)',
 'Honey, I Shrunk the Kids (1989)',
 'Escape from New York (1981)',
 'Milk Money (1994)',
 'Forces of Nature (1999)',
 'Space Jam (1996)',
 'Dragonheart (1996)']

'Grumpy Old Men (1993)'을 추천하였다.

# 마무리

Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 CSR Matrix를 이용해 추천 모델을 만들어 추천시스템이 전반적으로 어떻게 돌아가는지 알게되었다. 

학습 된 데이터는 fitting 된 것 같은데 주어진 영화 목록 중에서 예측하는 능력은 많이 떨어지는 것으로 보인다. 이는 좋아하는 영화 목록에 일관성이 없었기 때문이 아닌가 생각된다. 사실 주어진 평점하고 영화 제목만 가지고 학습을 진행하였으니 선호도를 정확히 판단하는 것은 어려울 것이다.

장르 데이터도 있었는데. 이에 대한 가중치를 적용하여 학습하면 더 좋은 정확도를 보여줄 수 있지 않을까 생각된다. 